# Pulling Venue Data from FourSquare API

__Skip to the Good Stuff__

[Importing Libraries](#Importing-Libraries)

[Connecting to FourSquare](#Connecting-to-FourSquare)

[Testing the Code](#Testing-the-Code)

[Making the DataFrame](#Making-the-DataFrame)

[Dropping the Nulls](#Dropping-the-Nulls)

[Sorting by Venue and Distance](#Sorting-by-Venue-and-Distance)

[Obtaining Neighborhood for Each Arcade](#Obtaining-Neighborhood-for-Each-Arcade)

[Pulling the Ratings](#Pulling-the-Ratings)

### Importing Libraries

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

! pip install folium==0.5.0
import folium # plotting library

import json

pd.set_option('display.max_rows', None) #To view the full dataframe

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


In [3]:
dallas_df = pd.read_csv('dallas_geodata.csv')
dallas_df.head(10)

,Area,Neighborhood,area_longitude,area_latitude,hood_longitude,hood_latitude
0,Plano & North Dallas,Addison,-96.699857,32.9542425,-96.8300673063398,32.960477850000004
1,Plano & North Dallas,Argyle,-96.699857,32.9542425,-96.80766323647441,32.80973185
2,Plano & North Dallas,Bartonville,-96.699857,32.9542425,No Data Found,No Data Found
3,Plano & North Dallas,Celeste,-96.699857,32.9542425,-96.633491,32.886654
4,Plano & North Dallas,Coppell,-96.699857,32.9542425,-97.0155703,32.9552598
5,Plano & North Dallas,Corinth,-96.699857,32.9542425,-96.7985927994061,32.748239850000004
6,Plano & North Dallas,Double Oak,-96.699857,32.9542425,-97.0026044,32.827486
7,Plano & North Dallas,Farmers Branch,-96.699857,32.9542425,-96.8961151,32.9265137
8,Plano & North Dallas,Grapevine,-96.699857,32.9542425,-97.0320443,32.9271484
9,Plano & North Dallas,Greenville,-96.699857,32.9542425,-96.7701728,32.8146164


### Connecting to FourSquare

In [4]:
CLIENT_ID = '4TJ1SLTDO1KADL553GXCDXOJXZONZYTBIRQMVBQ32JYAU3EA' # your Foursquare ID
CLIENT_SECRET = 'TC1N45PW5LSGVLIZ02DDPA2JOMWB2I3BA33OPX55VW2LBKBF' # your Foursquare Secret
ACCESS_TOKEN = 'NS3WWURVAWWAGWSTUXQDQCS0150IPNQIYNHVE5JM1OSX2CAV' # your FourSquare Access Token

In [5]:
VERSION = '20180604'
LIMIT = 30

### Testing the Code

In [6]:
#assign variables for the search
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


In [7]:
for index, hood in enumerate(dallas_df['hood_latitude']):
    if index == 5: #preselected based on venues in initial search
        latitude= dallas_df['hood_latitude'][index]
        longitude= dallas_df['hood_longitude'][index]
print(latitude, longitude)

32.748239850000004 -96.7985927994061


In [8]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6141262a4ed5bf12c907d043'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': []}}

In [10]:
ident=results['response']['venues'][0]
type(ident)
for dictionary in results['response']['venues']:
    print(dictionary['id'])
    print(dictionary['name'])
    print(dictionary['location']['address'])
    print(dictionary['location']['lat'])
    print(dictionary['location']['lng'])
    print(dictionary['location']['distance'])
    for cat in dictionary['categories']:
        print(cat['id'], cat['name'])

IndexError: list index out of range

### Making the DataFrame

In [15]:
#assign variables for the search
search_query = 'Arcade'
radius = 2000
print(search_query + ' .... OK!')

Arcade .... OK!


In [16]:
dallas_venues = pd.DataFrame(columns = ['Hood','Venue_ID','Venue_Name','Venue_Distance','Venue_Address','Venue_Latitude','Venue_Longitude','Category_ID','Category_Name'])
dallas_venues.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name


In [17]:
for index, hood in enumerate(dallas_df['hood_latitude']):
    try:
        hood_name = dallas_df['Neighborhood'][index]
        latitude = dallas_df['hood_latitude'][index]
        longitude = dallas_df['hood_longitude'][index]
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
        results = requests.get(url).json()
    except:
        print('No Results', index)
        continue
    try:
        res = results['response']['venues']
    except:
        print('No Response', index)
        continue
    for dictionary in results['response']['venues']:
        try:
            ident= dictionary['id']
        except:
            ident= 'No ID'
        try:
            name = dictionary['name']
        except:
            name = 'No Name'
        try:
            ven_add = dictionary['location']['address']
        except:
            ven_add = 'No Address'
        try:
            ven_lat = dictionary['location']['lat']
        except:
            ven_lat = 'No Latitude'
        try:
            ven_lng = dictionary['location']['lng']
        except:
            ven_lng = 'No Longitude'
        try:
            ven_dist = dictionary['location']['distance']
        except:
            ven_dist = 'No Distance'
        try:
            for cat in dictionary['categories']:
                try:
                    cat_id= cat['id']
                except:
                    cat_id= 'No ID'
                try:
                    cat_name = cat['name']
                except:
                    cat_name = 'No Name'
        except:
                continue
        dallas_venues = dallas_venues.append({'Venue_Distance':ven_dist,'Hood':hood_name, 'Venue_ID':ident, 'Venue_Name':name,'Venue_Address':ven_add,'Venue_Latitude':ven_lat,'Venue_Longitude':ven_lng,'Category_ID':cat_id,'Category_Name':cat_name}, ignore_index=True)

No Response 2
No Response 26
No Response 32
No Response 38
No Response 42
No Response 49
No Response 53
No Response 55
No Response 62
No Response 63
No Response 64
No Response 75
No Response 76
No Response 77
No Response 78
No Response 82
No Response 84
No Response 89
No Response 94
No Response 102
No Response 110
No Response 118
No Response 122
No Response 128
No Response 134
No Response 139
No Response 140
No Response 141
No Response 142
No Response 143
No Response 144
No Response 145
No Response 146
No Response 147
No Response 148
No Response 149
No Response 150
No Response 151
No Response 152
No Response 153
No Response 154
No Response 155
No Response 156
No Response 157
No Response 158
No Response 159
No Response 160
No Response 161
No Response 162
No Response 163
No Response 164
No Response 165
No Response 166
No Response 167
No Response 168
No Response 169
No Response 170
No Response 171
No Response 172
No Response 173
No Response 174
No Response 175
No Response 176
No Response 

In [18]:
dallas_venues = dallas_venues.set_index('Hood')
print(len(dallas_venues))
dallas_venues

17


,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
Hood,,,,,,,,
Coppell,4f551efde4b0336536ffd4a6,Noah's Arcade,1520,No Address,32.964215,-97.027868,4bf58dd8d48988d1e1931735,Arcade
Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade
Greenville,4ab03c84f964a520996620e3,Arcade Bar,1602,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
Plano,56631de8498e2afbdc225b72,Free Play Arcade,1305,1730 E Belt Line Rd,32.943863,-96.693342,4bf58dd8d48988d1e1931735,Arcade
Sachse,574931cd498ec191ec0e8f52,Gonzo's Rolling Arcade,1665,No Address,32.961290,-96.594286,4bf58dd8d48988d1e1931735,Arcade
The Colony,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,1355,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot
Rockwall,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,840,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot
Arts District,50d39fe2e4b06054da0fc08b,BBs Arcade,905,412 N Tyler St,32.748692,-96.837542,4bf58dd8d48988d1e1931735,Arcade
Bishop Arts District,50d39fe2e4b06054da0fc08b,BBs Arcade,905,412 N Tyler St,32.748692,-96.837542,4bf58dd8d48988d1e1931735,Arcade


In [19]:
print(len(dallas_venues))
dallas_venues.to_csv('{}_venue_data.csv'.format(search_query))

17


### Dropping the Nulls

In [20]:
dallas_venues = pd.read_csv('{}_venue_data.csv'.format(search_query))
print(search_query)
dallas_venues.head()

Arcade


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Coppell,4f551efde4b0336536ffd4a6,Noah's Arcade,1520,No Address,32.964215,-97.027868,4bf58dd8d48988d1e1931735,Arcade
1,Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade
2,Greenville,4ab03c84f964a520996620e3,Arcade Bar,1602,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
3,Plano,56631de8498e2afbdc225b72,Free Play Arcade,1305,1730 E Belt Line Rd,32.943863,-96.693342,4bf58dd8d48988d1e1931735,Arcade
4,Sachse,574931cd498ec191ec0e8f52,Gonzo's Rolling Arcade,1665,No Address,32.961290,-96.594286,4bf58dd8d48988d1e1931735,Arcade


In [21]:
print(len(dallas_venues))

17


In [22]:
dallas_venues = dallas_venues[dallas_venues != 'Null']
dallas_venues = dallas_venues[dallas_venues != '[]']
dallas_venues = dallas_venues.dropna()
print(len(dallas_venues))
dallas_venues = dallas_venues.reset_index()
dallas_venues = dallas_venues.drop(['index'], axis = 1)
dallas_venues.head()

17


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Coppell,4f551efde4b0336536ffd4a6,Noah's Arcade,1520,No Address,32.964215,-97.027868,4bf58dd8d48988d1e1931735,Arcade
1,Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade
2,Greenville,4ab03c84f964a520996620e3,Arcade Bar,1602,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
3,Plano,56631de8498e2afbdc225b72,Free Play Arcade,1305,1730 E Belt Line Rd,32.943863,-96.693342,4bf58dd8d48988d1e1931735,Arcade
4,Sachse,574931cd498ec191ec0e8f52,Gonzo's Rolling Arcade,1665,No Address,32.961290,-96.594286,4bf58dd8d48988d1e1931735,Arcade


### Sorting by Venue and Distance

In [24]:
dallas_sorted = dallas_venues.sort_values(['Venue_ID','Venue_Distance'], ascending = True)
dallas_sorted.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
13,Swiss Avenue,4ab03c84f964a520996620e3,Arcade Bar,515,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
12,Munger Place,4ab03c84f964a520996620e3,Arcade Bar,1437,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
2,Greenville,4ab03c84f964a520996620e3,Arcade Bar,1602,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
11,Lower Greenville,4ab03c84f964a520996620e3,Arcade Bar,1602,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
10,Lakewood,4ab03c84f964a520996620e3,Arcade Bar,1661,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar


In [25]:
dallas_sorted['Category_Name'].unique()

array(['Bar', 'Arcade', 'Nightlife Spot'], dtype=object)

In [26]:
#Didn't drop any categories
#Can't conclude the categories aren't arcades
dallas_sorted['Category_Name'].unique()

array(['Bar', 'Arcade', 'Nightlife Spot'], dtype=object)

In [27]:
dallas_sorted.to_csv('{}_sorted.csv'.format(search_query))

### Obtaining Neighborhood for Each Arcade

In [28]:
dallas_sorted = pd.read_csv('{}_sorted.csv'.format(search_query))

In [29]:
dallas_sorted = dallas_sorted.drop('Unnamed: 0',axis =1)

In [30]:
print(len(dallas_sorted))
dallas_sorted

17


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Swiss Avenue,4ab03c84f964a520996620e3,Arcade Bar,515,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
1,Munger Place,4ab03c84f964a520996620e3,Arcade Bar,1437,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
2,Greenville,4ab03c84f964a520996620e3,Arcade Bar,1602,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
3,Lower Greenville,4ab03c84f964a520996620e3,Arcade Bar,1602,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
4,Lakewood,4ab03c84f964a520996620e3,Arcade Bar,1661,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
5,Garrett,4ab03c84f964a520996620e3,Arcade Bar,1764,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
6,Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade
7,Bluffview,4e18b36c183880768f442784,The Arcade,1687,No Address,32.970596,-96.925092,4bf58dd8d48988d1e1931735,Arcade
8,Rockwall,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,840,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot
9,The Colony,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,1355,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot


In [31]:
count = -1
list_to_drop = list()
for ID in dallas_sorted['Venue_ID']:
    count = count +1
    try:
        if dallas_sorted['Venue_ID'][count] == dallas_sorted['Venue_ID'][count-1] and dallas_sorted['Venue_Distance'][count]>=dallas_sorted['Venue_Distance'][count-1]:
            list_to_drop.append(count)
    except:
        continue
list_to_drop

[1, 2, 3, 4, 5, 9, 12, 13, 14]

In [32]:
dallas_final = dallas_sorted.drop(index = list_to_drop)

In [33]:
print(len(dallas_final))
dallas_final

8


,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Swiss Avenue,4ab03c84f964a520996620e3,Arcade Bar,515,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
6,Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade
7,Bluffview,4e18b36c183880768f442784,The Arcade,1687,No Address,32.970596,-96.925092,4bf58dd8d48988d1e1931735,Arcade
8,Rockwall,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,840,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot
10,Coppell,4f551efde4b0336536ffd4a6,Noah's Arcade,1520,No Address,32.964215,-97.027868,4bf58dd8d48988d1e1931735,Arcade
11,Arts District,50d39fe2e4b06054da0fc08b,BBs Arcade,905,412 N Tyler St,32.748692,-96.837542,4bf58dd8d48988d1e1931735,Arcade
15,Plano,56631de8498e2afbdc225b72,Free Play Arcade,1305,1730 E Belt Line Rd,32.943863,-96.693342,4bf58dd8d48988d1e1931735,Arcade
16,Sachse,574931cd498ec191ec0e8f52,Gonzo's Rolling Arcade,1665,No Address,32.961290,-96.594286,4bf58dd8d48988d1e1931735,Arcade


In [34]:
dallas_final.to_csv('{}_unique.csv'.format(search_query))

### Pulling the Ratings

In [35]:
search_query

'Arcade'

In [36]:
dallas_final = pd.read_csv('{}_unique.csv'.format(search_query))
dallas_final.head()

,Unnamed: 0,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,0,Swiss Avenue,4ab03c84f964a520996620e3,Arcade Bar,515,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
1,6,Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade
2,7,Bluffview,4e18b36c183880768f442784,The Arcade,1687,No Address,32.970596,-96.925092,4bf58dd8d48988d1e1931735,Arcade
3,8,Rockwall,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,840,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot
4,10,Coppell,4f551efde4b0336536ffd4a6,Noah's Arcade,1520,No Address,32.964215,-97.027868,4bf58dd8d48988d1e1931735,Arcade


In [37]:
dallas_final = dallas_final.drop('Unnamed: 0', axis = 1)
dallas_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name
0,Swiss Avenue,4ab03c84f964a520996620e3,Arcade Bar,515,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar
1,Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade
2,Bluffview,4e18b36c183880768f442784,The Arcade,1687,No Address,32.970596,-96.925092,4bf58dd8d48988d1e1931735,Arcade
3,Rockwall,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,840,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot
4,Coppell,4f551efde4b0336536ffd4a6,Noah's Arcade,1520,No Address,32.964215,-97.027868,4bf58dd8d48988d1e1931735,Arcade


In [37]:
dallas_final = dallas_final.assign(Venue_Rating='NaN')
dallas_final

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,University District,4af30888f964a520f5e921e3,4518 Arcade,59,4518 University Way NE,47.661836,-122.313121,4bf58dd8d48988d124941735,Office,NaN
1,Lower Queen Anne,4d03d2b6c2e537042963b467,Penny Arcade Warehouse,473,3150 Elliott Ave,47.618242,-122.357470,4bf58dd8d48988d1e1931735,Arcade,NaN
2,Highland Park,4dbf6f1e6a23e5a549d6141b,SanTe's Arcade,752,No Address,47.524985,-122.361241,4bf58dd8d48988d1e1931735,Arcade,NaN
3,Pioneer Square,4e53fe8522711d4da5c38798,Grand Central Building - Lower Arcade,186,No Address,47.600463,-122.333970,5744ccdfe4b0c0459246b4dc,Shopping Plaza,NaN
4,Fremont,4ee59ef76c25be9631860f8d,Add-a-Ball Arcade,399,315 N 36th St #2B,47.652040,-122.354771,4bf58dd8d48988d1e1931735,Arcade,NaN
5,Pioneer Square,4fb7f757e4b09a1e8cab118e,Time masheen arcade,252,No Address,47.601545,-122.330684,4bf58dd8d48988d1e1931735,Arcade,NaN
6,Lower Queen Anne,4fdd404be4b082d0767d2155,Seattle Interactive Media Museum,387,No Address,47.621242,-122.350853,4bf58dd8d48988d1e1931735,Arcade,NaN
7,Renton Hill,4ff503b1e4b02ca33b4b731f,Rod's Pinball Arcade,829,9818 Beacon Ave S,47.512992,-122.272850,4bf58dd8d48988d1e1931735,Arcade,NaN
8,Lower Queen Anne,5064c506e4b06062642548ae,Next 50 Video Arcade,390,Seattle Center,47.621220,-122.350827,4bf58dd8d48988d1e1931735,Arcade,NaN
9,Harrison,50fa31bae4b020b5559a65cc,Lipton's Arcade,373,No Address,47.618770,-122.326777,4bf58dd8d48988d11a941735,Other Nightlife,NaN


In [38]:
count = -1
rating_list = list()
for ident in dallas_final['Venue_ID']:
    count = count +1
    venue_id = dallas_final['Venue_ID'][count]
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
    result = requests.get(url).json()
    try:
        rating = result['response']['venue']['rating']
    except:
        rating = 'Not Rated'
    rating_list.append(rating)
rating_list

['Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated',
 'Not Rated']

In [40]:
dallas_final = dallas_final.assign(Venue_Rating = rating_list)

In [41]:
dallas_final.head()

,Hood,Venue_ID,Venue_Name,Venue_Distance,Venue_Address,Venue_Latitude,Venue_Longitude,Category_ID,Category_Name,Venue_Rating
0,Swiss Avenue,4ab03c84f964a520996620e3,Arcade Bar,515,1825 Abrams Rd,32.812223,-96.753279,4bf58dd8d48988d116941735,Bar,Not Rated
1,Grapevine,4c538a9530f92d7fad2af0b8,Northern Lights Arcade at Great Wolf Lodge,905,100 Great Wolf Dr,32.932259,-97.039587,4bf58dd8d48988d1e1931735,Arcade,Not Rated
2,Bluffview,4e18b36c183880768f442784,The Arcade,1687,No Address,32.970596,-96.925092,4bf58dd8d48988d1e1931735,Arcade,Not Rated
3,Rockwall,4f32bb1b19836c91c7f4450e,Spin and Win Arcade,840,10219 Plano Rd,32.889265,-96.703578,4d4b7105d754a06376d81259,Nightlife Spot,Not Rated
4,Coppell,4f551efde4b0336536ffd4a6,Noah's Arcade,1520,No Address,32.964215,-97.027868,4bf58dd8d48988d1e1931735,Arcade,Not Rated


In [42]:
dallas_final.to_csv('{}_final_clean.csv'.format(search_query))